London Derby: Comparison of social media reach and sentiment between Arsenal FC and Tottenham FC

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize

import re
import string
from collections import Counter
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer , ENGLISH_STOP_WORDS
from wordcloud import WordCloud
from textblob import TextBlob
from collections import Counter
import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, ols
from scipy import stats

These are the libraries I utilized for the project

In [92]:
df_arsenal = pd.read_csv('arsenal.csv')
df_tottenham = pd.read_csv('evergrand_tweets.csv')

Importing of the two dataframes

In [93]:
df_arsenal.head()

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,user,followers,friends,hashtags,place,coordinates,favorites
0,1497686791742238725,Sat Feb 26 21:35:31 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @NSftbl: [THREAD] Some insight &amp; analys...,THREAD Some insight amp analysis Mikel Arteta ...,"Sentiment(polarity=0.18333333333333338, subjec...",0.183333,0.500,en,0,43,"{'id': 1357429830388764673, 'id_str': '1357429...",705,315,NaN,NSftbl,Earth,NaN
1,1497686787229163528,Sat Feb 26 21:35:30 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @PatrickTimmons1: Spurs losing on Wednesday...,Spurs losing Wednesday Arsenal th minute winne...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000,en,0,783,"{'id': 294351394, 'id_str': '294351394', 'name...",335,205,NaN,PatrickTimmons1,London,NaN
2,1497686778207109123,Sat Feb 26 21:35:28 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@grumps1000x @premierleague @ChelseaFC @NUFC @...,Okay okay I guess I said …,"Sentiment(polarity=0.5, subjectivity=0.5)",0.500000,0.500,en,0,37,"{'id': 1571407892, 'id_str': '1571407892', 'na...",407,1561,NaN,"grumps1000x, premierleague, ChelseaFC, NUFC, M...","Santa Cruz de Miramar, Nayarit",NaN
3,1497686762747047938,Sat Feb 26 21:35:24 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @pinkygreenfpl: Yes your FH team got more p...,Yes FH team got points non FH team fucking wil...,"Sentiment(polarity=-0.6, subjectivity=0.8)",-0.600000,0.800,en,0,2,"{'id': 1323553285064765440, 'id_str': '1323553...",8616,931,NaN,pinkygreenfpl,"Amalfi, Campania",NaN
4,1497686758540001280,Sat Feb 26 21:35:23 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @KyleWOrton: #pt Russia has its full milita...,Russia full military arsenal move Kiev tanks a...,"Sentiment(polarity=0.12499999999999999, subjec...",0.125000,0.325,en,0,6,"{'id': 1063731250500915200, 'id_str': '1063731...",534,857,pt,KyleWOrton,United Kingdom,NaN


In [94]:
df_tottenham.head()

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,user,followers,friends,hashtags,place,coordinates,favorites
0,1443713386361937923,Thu Sep 30 23:04:28 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",RT @idesibanda: 128+ Banks &amp; 121 NBFCs are...,Banks amp NBFCs eed liabilities,"Sentiment(polarity=0.0, subjectivity=0.0)",0.00,0.000,en,0,54,"{'id': 1414543824920059907, 'id_str': '1414543...",18,10,"Evergrande, EvergrandeCrisis",idesibanda,NaN,NaN
1,1443702303509434374,Thu Sep 30 22:20:26 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",RT @RedRockSec: Is #gold a safe investment dur...,Is safe investment debt crisis …,"Sentiment(polarity=0.5, subjectivity=0.5)",0.50,0.500,en,0,10,"{'id': 2478669578, 'id_str': '2478669578', 'na...",7874,1468,"gold, Evergrande, EvergrandeCrisis, Evergrande...",RedRockSec,USA,NaN
2,1443697168301309955,Thu Sep 30 22:00:02 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Evergrande: Chinese real estate giant misses i...,Evergrande Chinese real estate giant misses in...,"Sentiment(polarity=0.05, subjectivity=0.325)",0.05,0.325,en,1,0,"{'id': 1057148786189824000, 'id_str': '1057148...",2818,21,NaN,NaN,NaN,NaN
3,1443684577273204736,Thu Sep 30 21:10:00 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",#Chinese property giant #Evergrande has missed...,property giant missed interest payments overse...,"Sentiment(polarity=0.0, subjectivity=0.5)",0.00,0.500,en,1,0,"{'id': 1297909248731385856, 'id_str': '1297909...",1098,1,"Chinese, Evergrande",NaN,NaN,NaN
4,1443638829815156747,Thu Sep 30 18:08:13 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @GordonGChang: #China’s debt-laden property...,’ debt-laden property sector—and perhaps econo...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.00,0.000,en,0,82,"{'id': 1113159183946452993, 'id_str': '1113159...",239,248,"China, Chinese, Evergrande",GordonGChang,NaN,NaN


Info on the head and tail of the dataset

In [95]:
df_arsenal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2615 entries, 0 to 2614
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2615 non-null   int64  
 1   created_at      2615 non-null   object 
 2   source          2615 non-null   object 
 3   original_text   2615 non-null   object 
 4   clean_text      2615 non-null   object 
 5   sentiment       2615 non-null   object 
 6   polarity        2615 non-null   float64
 7   subjectivity    2615 non-null   float64
 8   lang            2615 non-null   object 
 9   favorite_count  2615 non-null   int64  
 10  retweet_count   2615 non-null   int64  
 11  user            2615 non-null   object 
 12  followers       2615 non-null   int64  
 13  friends         2615 non-null   int64  
 14  hashtags        250 non-null    object 
 15  place           2205 non-null   object 
 16  coordinates     1642 non-null   object 
 17  favorites       12 non-null     o

In [96]:
df_arsenal.describe()

,id,polarity,subjectivity,favorite_count,retweet_count,followers,friends
count,2.615000e+03,2615.000000,2615.000000,2615.000000,2615.000000,2615.000000,2615.000000
mean,1.497674e+18,0.086713,0.314543,1.206501,226.698662,2782.689866,1428.991969
std,7.326820e+12,0.295516,0.315890,12.578909,928.272383,16968.003947,2626.441127
min,1.497662e+18,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.497667e+18,0.000000,0.000000,0.000000,0.000000,117.000000,249.500000
50%,1.497674e+18,0.000000,0.267857,0.000000,2.000000,457.000000,642.000000
75%,1.497680e+18,0.200000,0.544444,0.000000,77.000000,1713.500000,1670.000000
max,1.497687e+18,1.000000,1.000000,372.000000,16974.000000,622221.000000,62263.000000


Summary Statistics on the Arsenal dataframe

In [97]:
df_tottenham.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2110 entries, 0 to 2109
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2110 non-null   int64  
 1   created_at      2110 non-null   object 
 2   source          2110 non-null   object 
 3   original_text   2110 non-null   object 
 4   clean_text      2110 non-null   object 
 5   sentiment       2110 non-null   object 
 6   polarity        2110 non-null   float64
 7   subjectivity    2110 non-null   float64
 8   lang            2110 non-null   object 
 9   favorite_count  2110 non-null   int64  
 10  retweet_count   2110 non-null   int64  
 11  user            2110 non-null   object 
 12  followers       2110 non-null   int64  
 13  friends         2110 non-null   int64  
 14  hashtags        1804 non-null   object 
 15  place           1581 non-null   object 
 16  coordinates     1276 non-null   object 
 17  favorites       9 non-null      o

In [98]:
df_tottenham.describe()

,id,polarity,subjectivity,favorite_count,retweet_count,followers,friends
count,2.110000e+03,2110.000000,2110.000000,2110.000000,2110.000000,2.110000e+03,2110.000000
mean,1.441296e+18,-0.014279,0.285821,2.496209,68.061611,3.934290e+04,1381.812796
std,8.759598e+14,0.279394,0.339091,22.603846,92.299454,2.934513e+05,3251.413302
min,1.440224e+18,-1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,1.440628e+18,-0.019792,0.000000,0.000000,1.000000,1.080000e+02,146.250000
50%,1.441012e+18,0.000000,0.066667,0.000000,26.000000,3.660000e+02,437.500000
75%,1.441843e+18,0.000000,0.580357,0.000000,82.000000,1.579000e+03,1369.000000
max,1.443713e+18,1.000000,1.000000,630.000000,278.000000,5.801668e+06,68756.000000


Summary Statistics on the Tottenham dataframe

In [99]:
impressions_arsenal = df_arsenal["favorite_count"] + df_arsenal["retweet_count"]
df_arsenal["impressions"] = impressions_arsenal

reach = df_arsenal["followers"] + df_arsenal["friends"]
df_arsenal["reach"] = reach

engagement = df_arsenal["impressions"] / df_arsenal["reach"]
df_arsenal["engagement"] = engagement

a_engagement_rate = df_arsenal["engagement"]*100
df_arsenal["engagement_rate"] = a_engagement_rate

Calculating impressions, reach, engagement, and engagement rate for the Arsenal dataframe.

In [100]:
df_arsenal.head()

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,followers,friends,hashtags,place,coordinates,favorites,impressions,reach,engagement,engagement_rate
0,1497686791742238725,Sat Feb 26 21:35:31 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @NSftbl: [THREAD] Some insight &amp; analys...,THREAD Some insight amp analysis Mikel Arteta ...,"Sentiment(polarity=0.18333333333333338, subjec...",0.183333,0.500,en,0,...,705,315,NaN,NSftbl,Earth,NaN,43,1020,0.042157,4.215686
1,1497686787229163528,Sat Feb 26 21:35:30 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @PatrickTimmons1: Spurs losing on Wednesday...,Spurs losing Wednesday Arsenal th minute winne...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000,en,0,...,335,205,NaN,PatrickTimmons1,London,NaN,783,540,1.450000,145.000000
2,1497686778207109123,Sat Feb 26 21:35:28 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@grumps1000x @premierleague @ChelseaFC @NUFC @...,Okay okay I guess I said …,"Sentiment(polarity=0.5, subjectivity=0.5)",0.500000,0.500,en,0,...,407,1561,NaN,"grumps1000x, premierleague, ChelseaFC, NUFC, M...","Santa Cruz de Miramar, Nayarit",NaN,37,1968,0.018801,1.880081
3,1497686762747047938,Sat Feb 26 21:35:24 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @pinkygreenfpl: Yes your FH team got more p...,Yes FH team got points non FH team fucking wil...,"Sentiment(polarity=-0.6, subjectivity=0.8)",-0.600000,0.800,en,0,...,8616,931,NaN,pinkygreenfpl,"Amalfi, Campania",NaN,2,9547,0.000209,0.020949
4,1497686758540001280,Sat Feb 26 21:35:23 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @KyleWOrton: #pt Russia has its full milita...,Russia full military arsenal move Kiev tanks a...,"Sentiment(polarity=0.12499999999999999, subjec...",0.125000,0.325,en,0,...,534,857,pt,KyleWOrton,United Kingdom,NaN,6,1391,0.004313,0.431344


In [101]:
impressions_tottenham = df_tottenham["favorite_count"] + df_tottenham["retweet_count"]
df_tottenham["impressions"] = impressions_tottenham

t_reach = df_tottenham["followers"] + df_tottenham["friends"]
df_tottenham["reach"] = t_reach

t_engagement = df_tottenham["impressions"] / df_tottenham["reach"]
df_tottenham["engagement"] = t_engagement

t_engagement_rate = df_tottenham["engagement"]*100
df_tottenham["engagement_rate"] = t_engagement_rate

Adding same metrics that were used for Arsenal in the Tottenham dataframe

In [102]:
df_arsenal["Arsenal"] = 1
df_tottenham["Arsenal"]=0

Adding dummy variabeles to the twoframes so that once combined together the data can be filtered by Arsenal and Tottenham

In [103]:
frames = [df_arsenal, df_tottenham]

London_Clubs = pd.concat(frames)

Putting the two dataframes into a single set and then using and concat function to combine the two togther into a signle dataframe called London Clubs

In [104]:
London_Clubs.head()

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,friends,hashtags,place,coordinates,favorites,impressions,reach,engagement,engagement_rate,Arsenal
0,1497686791742238725,Sat Feb 26 21:35:31 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @NSftbl: [THREAD] Some insight &amp; analys...,THREAD Some insight amp analysis Mikel Arteta ...,"Sentiment(polarity=0.18333333333333338, subjec...",0.183333,0.500,en,0,...,315,NaN,NSftbl,Earth,NaN,43,1020,0.042157,4.215686,1
1,1497686787229163528,Sat Feb 26 21:35:30 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @PatrickTimmons1: Spurs losing on Wednesday...,Spurs losing Wednesday Arsenal th minute winne...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000,en,0,...,205,NaN,PatrickTimmons1,London,NaN,783,540,1.450000,145.000000,1
2,1497686778207109123,Sat Feb 26 21:35:28 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@grumps1000x @premierleague @ChelseaFC @NUFC @...,Okay okay I guess I said …,"Sentiment(polarity=0.5, subjectivity=0.5)",0.500000,0.500,en,0,...,1561,NaN,"grumps1000x, premierleague, ChelseaFC, NUFC, M...","Santa Cruz de Miramar, Nayarit",NaN,37,1968,0.018801,1.880081,1
3,1497686762747047938,Sat Feb 26 21:35:24 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @pinkygreenfpl: Yes your FH team got more p...,Yes FH team got points non FH team fucking wil...,"Sentiment(polarity=-0.6, subjectivity=0.8)",-0.600000,0.800,en,0,...,931,NaN,pinkygreenfpl,"Amalfi, Campania",NaN,2,9547,0.000209,0.020949,1
4,1497686758540001280,Sat Feb 26 21:35:23 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @KyleWOrton: #pt Russia has its full milita...,Russia full military arsenal move Kiev tanks a...,"Sentiment(polarity=0.12499999999999999, subjec...",0.125000,0.325,en,0,...,857,pt,KyleWOrton,United Kingdom,NaN,6,1391,0.004313,0.431344,1


In [105]:
London_Clubs.describe()

,id,polarity,subjectivity,favorite_count,retweet_count,followers,friends,impressions,reach,engagement,engagement_rate,Arsenal
count,4.725000e+03,4725.000000,4725.000000,4725.000000,4725.000000,4.725000e+03,4725.000000,4725.00000,4.725000e+03,4722.000000,4722.000000,4725.000000
mean,1.472498e+18,0.041614,0.301717,1.782434,155.857566,1.910905e+04,1407.923598,157.64000,2.051697e+04,inf,inf,0.553439
std,2.803647e+16,0.292737,0.326732,17.778344,697.736005,1.973186e+05,2921.864448,697.70178,1.973150e+05,NaN,NaN,0.497189
min,1.440224e+18,-1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.00000,0.000000e+00,0.000000,0.000000,0.000000
25%,1.441152e+18,0.000000,0.000000,0.000000,0.000000,1.130000e+02,190.000000,1.00000,3.950000e+02,0.000199,0.019877,0.000000
50%,1.497664e+18,0.000000,0.200000,0.000000,7.000000,4.120000e+02,560.000000,9.00000,1.150000e+03,0.006601,0.660073,1.000000
75%,1.497675e+18,0.136364,0.550000,0.000000,82.000000,1.673000e+03,1505.000000,83.00000,3.434000e+03,0.080603,8.060283,1.000000
max,1.497687e+18,1.000000,1.000000,630.000000,16974.000000,5.801668e+06,68756.000000,16974.00000,5.801915e+06,inf,inf,1.000000


In [106]:
London_Clubs.sort_values(by='impressions', ascending=False)

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,friends,hashtags,place,coordinates,favorites,impressions,reach,engagement,engagement_rate,Arsenal
807,1497678230719021061,Sat Feb 26 21:01:30 +0000 2022,"<a href=""http://twitter.com/download/android"" ...","RT @mrsorokaa: President, his Chief of Staff, ...",President Chief Staff Prime Minister head Serv...,"Sentiment(polarity=0.8, subjectivity=0.4)",0.800000,0.400000,en,0,...,5409,NaN,mrsorokaa,NaN,NaN,16974,10332,1.642857,164.285714,1
124,1497685520582905862,Sat Feb 26 21:30:28 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @Arsenal: NEVER. GIVE. UP 👊 https://t.co/pV...,NEVER GIVE UP,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,en,0,...,136,NaN,Arsenal,Nigeria,NaN,14525,399,36.403509,3640.350877,1
949,1497677032091701258,Sat Feb 26 20:56:44 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @Arsenal: NEVER. GIVE. UP 👊 https://t.co/pV...,NEVER GIVE UP,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,en,0,...,519,NaN,Arsenal,🇸🇦,NaN,14525,559,25.983900,2598.389982,1
1121,1497675540047831044,Sat Feb 26 20:50:48 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @KyivIndependent: ⚡️Russian artillery fire ...,️Russian artillery fire struck Kyiv 's childre...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,en,0,...,5409,NaN,KyivIndependent,NaN,NaN,13099,10332,1.267809,126.780875,1
240,1497684291345600515,Sat Feb 26 21:25:35 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @LacazetteAlex: ARSENAL ❤️♣️,ARSENAL ️️,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,en,0,...,499,NaN,LacazetteAlex,🇺🇬,NaN,6557,1356,4.835546,483.554572,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,1497680029530140674,Sat Feb 26 21:08:39 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",@spencermorgan93 Arsenal fan worries about Che...,Arsenal fan worries Chelsea classic,"Sentiment(polarity=0.16666666666666666, subjec...",0.166667,0.166667,en,0,...,984,NaN,spencermorgan93,"England, United Kingdom",NaN,0,1193,0.000000,0.000000,1
640,1497679950987546629,Sat Feb 26 21:08:20 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",“…not simply one more skirmish in the endless ...,“ …not simply one skirmish endless cable war w...,"Sentiment(polarity=-0.0625, subjectivity=0.553...",-0.062500,0.553571,en,0,...,1004,NaN,NaN,"NY, NY",NaN,0,9275,0.000000,0.000000,1
1611,1497670468366528514,Sat Feb 26 20:30:39 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@srivatsank24 @SteveHermon All I can think of ...,All I think points West Ham I 'm still fuming ...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,en,0,...,595,NaN,"srivatsank24, SteveHermon","Melbourne, Victoria",NaN,0,740,0.000000,0.000000,1
654,1497679824210448385,Sat Feb 26 21:07:50 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",@F1del14 Yeah. I watch most games. Feel sorr...,Yeah I watch games Feel sorry players supporte...,"Sentiment(polarity=-0.5, subjectivity=1.0)",-0.500000,1.000000,en,0,...,185,NaN,F1del14,"Sydney, New South Wales",NaN,0,4675,0.000000,0.000000,1


In [107]:
London_Clubs.sort_values(by='engagement_rate', ascending=False)

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,friends,hashtags,place,coordinates,favorites,impressions,reach,engagement,engagement_rate,Arsenal
1089,1440987378202599427,Thu Sep 23 10:32:17 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Leaked suspected internal meeting of Evergrand...,Leaked suspected internal meeting Evergrande r...,"Sentiment(polarity=0.1875, subjectivity=0.375)",0.187500,0.3750,en,3,...,0,NaN,NaN,NaN,NaN,5,0,inf,inf,0
1893,1497667743075934210,Sat Feb 26 20:19:49 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@kwekuedilson It is difficult as they have pla...,It difficult played matches points On hand ars...,"Sentiment(polarity=-0.5, subjectivity=1.0)",-0.500000,1.0000,en,1,...,0,NaN,kwekuedilson,NaN,NaN,1,0,inf,inf,1
864,1497677800072835075,Sat Feb 26 20:59:47 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @TrollFootball: Arsenal still in his blood ...,Arsenal still blood,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.0000,en,0,...,3,NaN,TrollFootball,NaN,NaN,5993,3,1997.666667,1.997667e+05,1
1987,1497666920677580803,Sat Feb 26 20:16:33 +0000 2022,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @GOATSTREAM1: Today RETWEET🔁to watch live ...,Today RETWEETto watch live stream Twitter 󠁧󠁢󠁥󠁮...,"Sentiment(polarity=0.13636363636363635, subjec...",0.136364,0.5000,en,0,...,1,NaN,GOATSTREAM1,NaN,NaN,1139,1,1139.000000,1.139000e+05,1
18,1497686632459390980,Sat Feb 26 21:34:53 +0000 2022,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @IanWright0: No More Red.\n\nEvery young pe...,No More Red Every young person deserves opport...,"Sentiment(polarity=-0.049999999999999996, subj...",-0.050000,0.3000,en,0,...,1,NaN,IanWright0,NaN,NaN,1037,1,1037.000000,1.037000e+05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244,1497674265843359750,Sat Feb 26 20:45:45 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@fxheemldn @Hugojha23 @Dingydunga @Lacazetteee...,You mean n't born ha…,"Sentiment(polarity=-0.3125, subjectivity=0.6875)",-0.312500,0.6875,en,0,...,11,NaN,"fxheemldn, Hugojha23, Dingydunga, Lacazetteeee...",NaN,NaN,0,11,0.000000,0.000000e+00,1
1966,1497667096213434375,Sat Feb 26 20:17:15 +0000 2022,"<a href=""http://twitter.com/download/android"" ...","@TheNuamz Arsenal fans no go make we hear ei, ...",Arsenal fans go make hear ei hm,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.0000,en,0,...,592,NaN,TheNuamz,NaN,NaN,0,912,0.000000,0.000000e+00,1
1383,1497672926933716992,Sat Feb 26 20:40:25 +0000 2022,"<a href=""https://mobile.twitter.com"" rel=""nofo...",What prompted Ukraine to give up its nuclear a...,What prompted Ukraine give nuclear arsenal via,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.0000,en,0,...,0,NaN,timesofindia,Chennai-India,NaN,0,0,NaN,NaN,1
869,1441315427510939657,Fri Sep 24 08:15:50 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",You could go broke because of some company you...,You could go broke company ’ never heard EVERG...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.0000,en,0,...,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0


In [108]:
Column_filter = London_Clubs.drop(columns=['favorites', 'hashtags'])

Favorties and Hashtags were uneeded variables, so the two were dropped from the dataframe. The new dataframe with the dropped columns was called Column_filter

In [109]:
Column_filter.head()

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,user,followers,friends,place,coordinates,impressions,reach,engagement,engagement_rate,Arsenal
0,1497686791742238725,Sat Feb 26 21:35:31 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @NSftbl: [THREAD] Some insight &amp; analys...,THREAD Some insight amp analysis Mikel Arteta ...,"Sentiment(polarity=0.18333333333333338, subjec...",0.183333,0.500,en,0,...,"{'id': 1357429830388764673, 'id_str': '1357429...",705,315,NSftbl,Earth,43,1020,0.042157,4.215686,1
1,1497686787229163528,Sat Feb 26 21:35:30 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @PatrickTimmons1: Spurs losing on Wednesday...,Spurs losing Wednesday Arsenal th minute winne...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000,en,0,...,"{'id': 294351394, 'id_str': '294351394', 'name...",335,205,PatrickTimmons1,London,783,540,1.450000,145.000000,1
2,1497686778207109123,Sat Feb 26 21:35:28 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@grumps1000x @premierleague @ChelseaFC @NUFC @...,Okay okay I guess I said …,"Sentiment(polarity=0.5, subjectivity=0.5)",0.500000,0.500,en,0,...,"{'id': 1571407892, 'id_str': '1571407892', 'na...",407,1561,"grumps1000x, premierleague, ChelseaFC, NUFC, M...","Santa Cruz de Miramar, Nayarit",37,1968,0.018801,1.880081,1
3,1497686762747047938,Sat Feb 26 21:35:24 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @pinkygreenfpl: Yes your FH team got more p...,Yes FH team got points non FH team fucking wil...,"Sentiment(polarity=-0.6, subjectivity=0.8)",-0.600000,0.800,en,0,...,"{'id': 1323553285064765440, 'id_str': '1323553...",8616,931,pinkygreenfpl,"Amalfi, Campania",2,9547,0.000209,0.020949,1
4,1497686758540001280,Sat Feb 26 21:35:23 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @KyleWOrton: #pt Russia has its full milita...,Russia full military arsenal move Kiev tanks a...,"Sentiment(polarity=0.12499999999999999, subjec...",0.125000,0.325,en,0,...,"{'id': 1063731250500915200, 'id_str': '1063731...",534,857,KyleWOrton,United Kingdom,6,1391,0.004313,0.431344,1


In [110]:
Na_filter = Column_filter.dropna()
Na_filter.head()

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,user,followers,friends,place,coordinates,impressions,reach,engagement,engagement_rate,Arsenal
0,1497686791742238725,Sat Feb 26 21:35:31 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @NSftbl: [THREAD] Some insight &amp; analys...,THREAD Some insight amp analysis Mikel Arteta ...,"Sentiment(polarity=0.18333333333333338, subjec...",0.183333,0.500,en,0,...,"{'id': 1357429830388764673, 'id_str': '1357429...",705,315,NSftbl,Earth,43,1020,0.042157,4.215686,1
1,1497686787229163528,Sat Feb 26 21:35:30 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @PatrickTimmons1: Spurs losing on Wednesday...,Spurs losing Wednesday Arsenal th minute winne...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000,en,0,...,"{'id': 294351394, 'id_str': '294351394', 'name...",335,205,PatrickTimmons1,London,783,540,1.450000,145.000000,1
2,1497686778207109123,Sat Feb 26 21:35:28 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@grumps1000x @premierleague @ChelseaFC @NUFC @...,Okay okay I guess I said …,"Sentiment(polarity=0.5, subjectivity=0.5)",0.500000,0.500,en,0,...,"{'id': 1571407892, 'id_str': '1571407892', 'na...",407,1561,"grumps1000x, premierleague, ChelseaFC, NUFC, M...","Santa Cruz de Miramar, Nayarit",37,1968,0.018801,1.880081,1
3,1497686762747047938,Sat Feb 26 21:35:24 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @pinkygreenfpl: Yes your FH team got more p...,Yes FH team got points non FH team fucking wil...,"Sentiment(polarity=-0.6, subjectivity=0.8)",-0.600000,0.800,en,0,...,"{'id': 1323553285064765440, 'id_str': '1323553...",8616,931,pinkygreenfpl,"Amalfi, Campania",2,9547,0.000209,0.020949,1
4,1497686758540001280,Sat Feb 26 21:35:23 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @KyleWOrton: #pt Russia has its full milita...,Russia full military arsenal move Kiev tanks a...,"Sentiment(polarity=0.12499999999999999, subjec...",0.125000,0.325,en,0,...,"{'id': 1063731250500915200, 'id_str': '1063731...",534,857,KyleWOrton,United Kingdom,6,1391,0.004313,0.431344,1


Next it was time to omit NA variables. these were dropped from the previous dataframed and saved into a new one called Na_filter

In [111]:
Na_filter.tail()

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,user,followers,friends,place,coordinates,impressions,reach,engagement,engagement_rate,Arsenal
2096,1440238508451319820,Tue Sep 21 08:56:33 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",RT @srajb: Will #Evergrande of China spoil the...,Will China spoil global IPO rally Will next To...,"Sentiment(polarity=-0.16666666666666666, subje...",-0.166667,0.166667,en,0,...,"{'id': 1284536141220208641, 'id_str': '1284536...",123,110,srajb,washim,32,233,0.137339,13.733906,0
2099,1440235239364648968,Tue Sep 21 08:43:33 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",RT @ETNOWlive: #Evergrande | Do you think the ...,Do think could turn 's Lehman Brothers moment,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,en,0,...,"{'id': 1085618459000610816, 'id_str': '1085618...",12427,419,ETNOWlive,"Mumbai, India",8,12846,0.000623,0.062276,0
2100,1440235178266218498,Tue Sep 21 08:43:19 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",RT @srajb: Will #Evergrande of China spoil the...,Will China spoil global IPO rally Will next To...,"Sentiment(polarity=-0.16666666666666666, subje...",-0.166667,0.166667,en,0,...,"{'id': 1366753592380596224, 'id_str': '1366753...",41,18,srajb,"Patna, India",32,59,0.542373,54.237288,0
2101,1440234088019869707,Tue Sep 21 08:38:59 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @Manisha3005: Evergrande Crisis also hits t...,Evergrande Crisis also hits Cryptocurrency mar...,"Sentiment(polarity=0.1, subjectivity=0.4)",0.100000,0.400000,en,0,...,"{'id': 88480624, 'id_str': '88480624', 'name':...",603,166,Manisha3005,The Netherlands,4,769,0.005202,0.520156,0
2106,1440228723102400517,Tue Sep 21 08:17:40 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",RT @muhiblog: 13 years apart. \n\nHistory repe...,years apart History repeats Humans rarely learn,"Sentiment(polarity=0.3, subjectivity=0.9)",0.300000,0.900000,en,0,...,"{'id': 1277359375942746112, 'id_str': '1277359...",81,486,muhiblog,Planet A,17,567,0.029982,2.998236,0


In [112]:
filtered_Clubs = Na_filter[Na_filter['reach'] != 0]
Filtered_Clubs = filtered_Clubs[filtered_Clubs['impressions'] != 0]

When running analysis, reach and impressions that contained 0 would give a inf or nan engagement. In order to avoid this I filtred the dataframes to only includ reach and impressions that were not equal to 0

In [124]:
Filtered_Clubs= Filtered_Clubs.rename(columns={'id': 'ID','created_at': 'Created At', 'source': 'Source', 'original_text': 'Original Text', 'clean_text': 'Clean Text', 'sentiment': 'Sentiment', 'polarity': 'Polarity', 'subjectivity':'Subjectivity','lang':'Language','favorite_count':'Favorite Count','retweet_count':'Retweet Count','user':'User','followers':'Followers','friends':'Friends','place':'Place','coordinates':'Coordinates','impressions':'Impressions','reach':'Reach','engagement':'Engagement','engagement_rate':'Engagement Rate'})

Renaming all the variables to be capitalzied and not included underscores

In [125]:
Filtered_Clubs.head()

,ID,Created At,Source,Original Text,Clean Text,Sentiment,Polarity,Subjectivity,Language,Favorite Count,...,User,Followers,Friends,Place,Coordinates,Impressions,Reach,Engagement,Engagement Rate,Arsenal
0,1497686791742238725,Sat Feb 26 21:35:31 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @NSftbl: [THREAD] Some insight &amp; analys...,THREAD Some insight amp analysis Mikel Arteta ...,"Sentiment(polarity=0.18333333333333338, subjec...",0.183333,0.500,en,0,...,"{'id': 1357429830388764673, 'id_str': '1357429...",705,315,NSftbl,Earth,43,1020,0.042157,4.215686,1
1,1497686787229163528,Sat Feb 26 21:35:30 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @PatrickTimmons1: Spurs losing on Wednesday...,Spurs losing Wednesday Arsenal th minute winne...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000,en,0,...,"{'id': 294351394, 'id_str': '294351394', 'name...",335,205,PatrickTimmons1,London,783,540,1.450000,145.000000,1
2,1497686778207109123,Sat Feb 26 21:35:28 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@grumps1000x @premierleague @ChelseaFC @NUFC @...,Okay okay I guess I said …,"Sentiment(polarity=0.5, subjectivity=0.5)",0.500000,0.500,en,0,...,"{'id': 1571407892, 'id_str': '1571407892', 'na...",407,1561,"grumps1000x, premierleague, ChelseaFC, NUFC, M...","Santa Cruz de Miramar, Nayarit",37,1968,0.018801,1.880081,1
3,1497686762747047938,Sat Feb 26 21:35:24 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @pinkygreenfpl: Yes your FH team got more p...,Yes FH team got points non FH team fucking wil...,"Sentiment(polarity=-0.6, subjectivity=0.8)",-0.600000,0.800,en,0,...,"{'id': 1323553285064765440, 'id_str': '1323553...",8616,931,pinkygreenfpl,"Amalfi, Campania",2,9547,0.000209,0.020949,1
4,1497686758540001280,Sat Feb 26 21:35:23 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @KyleWOrton: #pt Russia has its full milita...,Russia full military arsenal move Kiev tanks a...,"Sentiment(polarity=0.12499999999999999, subjec...",0.125000,0.325,en,0,...,"{'id': 1063731250500915200, 'id_str': '1063731...",534,857,KyleWOrton,United Kingdom,6,1391,0.004313,0.431344,1


In [116]:
Filtered_Clubs.sort_values(by='Reach', ascending=True)

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,user,followers,friends,place,coordinates,impressions,reach,engagement,engagement_rate,Arsenal
61,1443215810185547786,Wed Sep 29 14:07:17 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",RT @Century_Fin: Fears over the property devel...,Fears property developer crisis spooked global...,"Sentiment(polarity=0.0, subjectivity=0.2)",0.000000,0.200000,en,0,...,"{'id': 114432964, 'id_str': '114432964', 'name...",8,8,Century_Fin,mumbai,4,16,2.500000e-01,25.000000,0
1659,1440561581926977538,Wed Sep 22 06:20:20 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",RT @Century_Fin: Concerns on the fallout of Ch...,Concerns fallout China ’ Evergrande haunting g...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,en,0,...,"{'id': 114432964, 'id_str': '114432964', 'name...",8,8,Century_Fin,mumbai,1,16,6.250000e-02,6.250000,0
1715,1497669466527879172,Sat Feb 26 20:26:40 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",@SkySportsPL the reason the ref didn’t go to t...,reason ref ’ go monitor reason ref ’ go monito...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,en,0,...,"{'id': 2205441038, 'id_str': '2205441038', 'na...",9,8,SkySportsPL,"London, England",1943,17,1.142941e+02,11429.411765,1
604,1441758386752667662,Sat Sep 25 13:36:00 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",RT @callistemon_red: Red day because of #Everg...,Red day Are buying 'dip preparing worse,"Sentiment(polarity=-0.2, subjectivity=0.3)",-0.200000,0.300000,en,0,...,"{'id': 1437432630677409793, 'id_str': '1437432...",3,16,callistemon_red,中国地方,1,19,5.263158e-02,5.263158,0
1017,1441025062287527956,Thu Sep 23 13:02:02 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",RT @WallStreetSilv: They are trying very hard ...,They trying hard convince us That usually mean...,"Sentiment(polarity=-0.27083333333333337, subje...",-0.270833,0.395833,en,0,...,"{'id': 1203812619548708865, 'id_str': '1203812...",0,20,WallStreetSilv,"DKI Jakarta, Indonesia",33,20,1.650000e+00,165.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,1440730668963565572,Wed Sep 22 17:32:13 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","The #Evergrande situation is similar to a ""con...",The situation similar `` controlled detonation...,"Sentiment(polarity=0.0, subjectivity=0.2)",0.000000,0.200000,en,10,...,"{'id': 68927629, 'id_str': '68927629', 'name':...",1184955,313,devinamehra,Mumbai,12,1185268,1.012429e-05,0.001012,0
1642,1440580507566108680,Wed Sep 22 07:35:32 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Catch @DevinaMehra talk about the default of #...,Catch talk default impact Indian market 's top...,"Sentiment(polarity=0.5, subjectivity=0.5)",0.500000,0.500000,en,7,...,"{'id': 68927629, 'id_str': '68927629', 'name':...",1184955,313,"devinamehra, firstglobalsec",Mumbai,7,1185268,5.905837e-06,0.000591,0
904,1441227134463143939,Fri Sep 24 02:25:00 +0000 2021,"<a href=""https://about.twitter.com/products/tw...",How does @NileshShah68 see China's Evergrande ...,How see China 's Evergrande crisis affecting I...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,en,4,...,"{'id': 43855487, 'id_str': '43855487', 'name':...",2137793,439,NileshShah68,India,4,2138232,1.870704e-06,0.000187,0
407,1442002804675993600,Sun Sep 26 05:47:14 +0000 2021,"<a href=""https://about.twitter.com/products/tw...",RT @ieexplained: What went wrong with #Evergra...,What went wrong 3/5,"Sentiment(polarity=-0.5, subjectivity=0.9)",-0.500000,0.900000,en,0,...,"{'id': 38647512, 'id_str': '38647512', 'name':...",3994916,315,ieexplained,India,4,3995231,1.001194e-06,0.000100,0


In [135]:
Summary_Statistics=Filtered_Clubs.describe()
Summary_Statistics.round(2)

,ID,Polarity,Subjectivity,Favorite Count,Retweet Count,Followers,Friends,Impressions,Reach,Engagement,Engagement Rate,Arsenal
count,1.878000e+03,1878.00,1878.00,1878.00,1878.00,1878.00,1878.00,1878.00,1878.00,1878.00,1878.00,1878.00
mean,1.471630e+18,0.04,0.31,0.47,223.69,12408.66,1919.49,224.16,14328.15,0.43,43.32,0.54
std,2.813616e+16,0.30,0.33,3.41,823.29,153627.68,3718.49,823.17,153706.61,4.55,455.39,0.50
min,1.440229e+18,-1.00,0.00,0.00,0.00,0.00,0.00,1.00,16.00,0.00,0.00,0.00
25%,1.441050e+18,0.00,0.00,0.00,3.00,205.25,331.75,5.00,651.50,0.00,0.31,0.00
50%,1.497663e+18,0.00,0.20,0.00,43.00,641.00,857.00,44.00,1714.00,0.02,2.22,1.00
75%,1.497674e+18,0.16,0.60,0.00,185.00,1997.75,2061.50,185.00,4375.00,0.11,10.65,1.00
max,1.497687e+18,1.00,1.00,84.00,14525.00,3994916.00,68756.00,14525.00,3995231.00,146.17,14617.07,1.00


In [127]:
model1 = ols("Polarity ~ Arsenal + Reach + Impressions + Engagement", Filtered_Clubs).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Polarity   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     14.37
Date:                Thu, 09 May 2024   Prob (F-statistic):           1.48e-11
Time:                        15:17:27   Log-Likelihood:                -345.54
No. Observations:                1878   AIC:                             701.1
Df Residuals:                    1873   BIC:                             728.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0119      0.010     -1.190      0.234      -0.031       0.008
Arsenal         0.1017      0.014      7.434      0.000       0.075       0.129
Reach       -3.083e-08   4.38e-08     -0.703      0.482   -1.17e-07    5.51e-08
Impressions -1.648e-05   9.04e-06     -1.824      0.068   -3.42e-05    1.24e-06
Engagement     -0.0002      0.002     -0.130      0.896      -0.003       0.003
==============================================================================
Omnibus:                       93.646   Durbin-Watson:                   1.717
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              153.633
Skew:                           0.406   Prob(JB):                     4.35e-34
Kurtosis:                       4.142   Cond. No.                     3.63e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.63e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

A regression ran using the key variables from the original research question

In [128]:
Arsenal_Club = Filtered_Clubs[Filtered_Clubs['Arsenal'] == 1]
Tottenham_Club = Filtered_Clubs[Filtered_Clubs['Arsenal'] == 0]

Seprating the two dataframes back into their original Arsenal and Tottenham data in order to run hypothesis testing

In [130]:
u_statistic, p_value = stats.mannwhitneyu(Arsenal_Club['Engagement'],Tottenham_Club['Engagement'])
print('U-Statistic: ', u_statistic)
print('p-value: ', p_value)

U-Statistic:  433989.0
p-value:  0.7149284528165663


In [131]:
u_statistic, p_value = stats.mannwhitneyu(Arsenal_Club['Polarity'],Tottenham_Club['Polarity'])
print('U-Statistic: ', u_statistic)
print('p-value: ', p_value)

U-Statistic:  524493.5
p-value:  5.9423182669528096e-15


In [122]:
Filtered_Clubs.to_csv('output.csv', index=False)

Exporting to open in Tableau